#Header

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import auth 
auth.authenticate_user() 
 
import gspread 
from google.auth import default 
creds, _ = default() 
 
gc = gspread.authorize(creds)

#Metrices

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def permanceMetrics(Y, pred, pred_prob):
    acc = accuracy_score(Y,pred)
    pre = precision_score(Y,pred,average='weighted')
    re = recall_score(Y,pred, average='weighted')
    f1 = f1_score(Y,pred, average='weighted')
    acrc = roc_auc_score(Y,pred_prob,multi_class= 'ovr')
    return acc, pre, re, f1, acrc

def avgMetric(met):
    res = np.array(met)
    acc = res[::5].mean()
    pre = res[1::5].mean()
    re = res[2::5].mean()
    f1 = res[3::5].mean()
    acrc = res[4::5].mean()
    return np.array([acc, pre, re, f1, acrc])

#Preprocess

In [ ]:
import re
def step03(data):
    step03_data = []
    for question in data:
        codeless_question=re.sub('<code>[^>]*</code>', '', question)
        tagless_question=re.sub('<[^>]*>', '', codeless_question)
        step03_data.append(tagless_question)
    return step03_data



In [ ]:
def remove_punctuation(data):
  step03_data=step03(data)
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  punctuationless_data = []
  for question in step03_data:
      no_punct = ""
      for char in question:
          if char not in punctuations:
              no_punct = no_punct + char
      punctuationless_data.append(no_punct)
  return punctuationless_data

# print(punctuationless_data[2])

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer


def step01(data):
  snow_stemmer = SnowballStemmer(language='english')

  punctuationless_data=remove_punctuation(data)
  step01_data = []
  for question in punctuationless_data:
    tokenized_question = []
    tokenized_question = word_tokenize(question)
    stem_question = []
    for t_ques in tokenized_question:
      stem_word = snow_stemmer.stem(t_ques)
      stem_question.append(stem_word)
    step01_data.append(stem_question)
  return step01_data

# print(step01_data[2])

In [ ]:
f=open('/content/drive/MyDrive/papers on stack overflow/User Expertise and Question Difficulty/Implementation/Stopwords.txt','r')
stop_words=list(f.read().split("\n"))

In [ ]:
def preprocessing(data):
  stop_words2 = nltk.corpus.stopwords.words('english')
  final_stopword_list = stop_words + stop_words2

  step01_data=step01(data)
  step02_data = []
  for question in step01_data:
    filtered_sentence=[]
    for word in question:
      if word not in final_stopword_list:
          filtered_sentence.append(word)
    step02_data.append(filtered_sentence)
  return step02_data 

# print(step02_data[2])

#Tf-Idf


In [ ]:
from collections import Counter
import math

def tf_idf(preprocessed):
    term_count=[]
    tf=[]
    #tf calculation
    for word in preprocessed:
        counts = Counter(word)
        count_list = [(k, v) for k, v in counts.items()]
        term_count.append(count_list)
    for doc in term_count:
        total_word=0
        for word in doc:
            total_word=total_word+word[1]
        tf_list=[]
        for word in doc:
            temp=word[1]/ float(total_word)
            tf_list.append((word[0],temp))
        tf.append(tf_list)
    #df calculation
    df=dict()
    for count_list in term_count:
        for word in count_list:
            if word[0] in df:
                df[word[0]]=df[word[0]]+word[1]
            else:
                df[word[0]]=word[1]
    deleting_key=[]
    for k in df.keys():
        if df[k]<3:
            deleting_key.append(k)
    for k in deleting_key:
        del df[k]

    #idf calculation
    idf = dict()
    for word in df:
        idf[word] = math.log(len(term_count) / float(df[word]))
    #tf-idf calculation
    tfidf_list=[]
    for count_list in tf:
        document=[]
        for word_list in count_list:
            word=word_list[0]
            if word_list[0] in idf:
                tfidf=word_list[1]*idf[word_list[0]]
            else:
                tfidf=word_list[1]*0
            tuple=(word,tfidf)
            document.append(tuple)
        tfidf_list.append(document)
    return tfidf_list,df

In [ ]:
def vectorizer_tfidf(processed_data):
    tfidf_list,df=tf_idf(processed_data)
    features=[x for x in df.keys()]
    tfidf_vector=[]
    for doc in tfidf_list:
        doc_wise_tfidf=[]
        for word in features:
            for term in doc:
                value=0
                if word==term[0]:
                    value=term[1]
                    break
            doc_wise_tfidf.append(value)
        tfidf_vector.append(doc_wise_tfidf)
    tfidf_vector_df=pd.DataFrame([[y for y in  x] for x in tfidf_vector],columns=features)
    return tfidf_vector_df

#Data Read

In [ ]:
import pandas as pd
worksheet1 = gc.open('Final Dataset').get_worksheet(0)
worksheet2 = gc.open('Final Dataset').get_worksheet(1)

rows1 = worksheet1.get_all_values()
rows2 = worksheet2.get_all_values()

stackData1=pd.DataFrame.from_records(rows1,columns=rows1[0])
stackData2=pd.DataFrame.from_records(rows2,columns=rows2[0])

stackData1.drop(0, inplace=True, axis=0)
stackData2.drop(0, inplace=True, axis=0)

stackData=[stackData1, stackData2]
stackData=pd.concat(stackData,ignore_index=True)

feature=["Body","QuestionLength", "Reputation","First_answer_Interval","favorite_count","question_score","view_count","answer_count"]

stackData["view_count"] = pd.to_numeric(stackData["view_count"])
stackData["answer_count"] = pd.to_numeric(stackData["answer_count"])
stackData["favorite_count"] = pd.to_numeric(stackData["favorite_count"])
stackData["question_score"] = pd.to_numeric(stackData["question_score"])

stackData["creation_date"] = pd.to_datetime(stackData["creation_date"],unit='s')
stackData["First_answer_date"] = pd.to_datetime(stackData["First_answer_date"],unit='s')
stackData["First_answer_Interval"]=(stackData["First_answer_date"]-stackData["creation_date"])/pd.Timedelta(minutes=1)
stackData["First_answer_Interval"] = stackData["First_answer_Interval"].apply(lambda x: -1 if x <= 0 else x)
# stackData["Accept_answer_Interval"]=stackData["Accept_answer_Interval"].apply(lambda x: -1 if x <= 0 else x)

data=stackData[feature].values
y=stackData.Label.values
print(len(data))

#Evaluation

In [ ]:
processed_data=preprocessing(data[:,0])
tfidf_vector_df=vectorizer_tfidf(processed_data)
print(tfidf_vector_df.shape)

other_features=data[:,1:8]
other_features_df=pd.DataFrame([[x for x in col] for col in other_features],columns=feature[1:8])
print(other_features_df.shape)

all_features_df=pd.concat([tfidf_vector_df,other_features_df], axis=1)
print(all_features_df.shape)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import confusion_matrix

skf = StratifiedKFold(n_splits=10)

i=0

all_features_df=np.array(all_features_df)


rm = []
print('Metrics :  Accuracy \t\tPrecision \t\tRecall \t\tF1-score \tAUC_ROC')
for train_index, test_index in skf.split(all_features_df,y):
    i=i+1
    X_train, X_test = all_features_df[train_index], all_features_df[test_index]
    y_train, y_test = y[train_index], y[test_index]

    rf = RF(n_estimators = 15, max_depth = 8, criterion='entropy', random_state = 42)
    rf.fit(X_train,y_train)
    pred = rf.predict(X_test)
    pred_prob = rf.predict_proba(X_test)
    acc, pre, re, f1, arc = permanceMetrics(y_test, pred, pred_prob)
    print('Fold-',i,': ', acc, pre, re, f1,arc)
    rm += [acc, pre, re, f1,arc]
    cm = confusion_matrix(y_test, pred)
    print(cm)
acc, pre, re, f1, acrc = avgMetric(rm)
print('\nAverage: ', acc, pre, re, f1, acrc)